# Johdanto todennäköisyyksiin ja tilastotieteeseen
Tässä muistikirjassa leikitellään joillakin aiemmin käsitellyillä käsitteillä. Monet todennäköisyyden ja tilastotieteen käsitteet ovat hyvin edustettuina suurissa Pythonin tietojenkäsittelyyn tarkoitetuissa kirjastoissa, kuten `numpy` ja `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Satunnaismuuttujat ja jakaumat
Aloitetaan piirtämällä otos, jossa on 30 arvoa yhtenäisestä jakaumasta välillä 0–9. Laskemme myös keskiarvon ja varianssin.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Arvioidaksemme visuaalisesti, kuinka monta erilaista arvoa otoksessa on, voimme piirtää **histogrammin**:


In [ ]:
plt.hist(sample)
plt.show()

## Todellisen datan analysointi

Keskiarvo ja varianssi ovat erittäin tärkeitä todellisen elämän datan analysoinnissa. Ladataan baseball-pelaajia koskeva data osoitteesta [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Käytämme täällä tietojenkäsittelyyn pakettia nimeltä [**Pandas**](https://pandas.pydata.org/). Puhumme lisää Pandasista ja datan käsittelystä Pythonilla myöhemmin tässä kurssissa.

Lasketaan keskiarvot iälle, pituudelle ja painolle:


In [ ]:
df[['Age','Height','Weight']].mean()

Keskitytään nyt pituuteen ja lasketaan keskipoikkeama ja varianssi:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Keskiarvon lisäksi on järkevää tarkastella mediaaniarvoa ja kvartiileja. Ne voidaan visualisoida käyttämällä **laatikko-plotkia**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Voimme myös tehdä laatikkokaavioita datamme osajoukoista, esimerkiksi pelaajarooleittain ryhmiteltynä.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Huomautus**: Tämä kaavio antaa ymmärtää, että keskimäärin ensimmäisen pesän pelaajien pituudet ovat pidempiä kuin toisen pesän pelaajien pituudet. Myöhemmin opimme, miten voimme testata tätä hypoteesia muodollisemmin ja miten voimme osoittaa, että datamme on tilastollisesti merkittävää tämän näyttämiseksi.

Ikä, pituus ja paino ovat kaikki jatkuvia satunnaismuuttujia. Mitä luulet niiden jakauman olevan? Hyvä tapa selvittää se on piirtää arvojen histogrammi:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normaali jakauma

Luodaan keinotekoinen painonäyte, joka noudattaa normaalia jakaumaa samalla keskiarvolla ja varianssilla kuin todelliset tietomme:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Koska useimmat arvot todellisessa elämässä ovat normaalijakautuneita, emme pitäisi käyttää tasaista satunnaislukugeneraattoria näytedata tuottamiseen. Tässä mitä tapahtuu, jos yritämme generoida painoja tasaiseen jakaumaan perustuen (generoitu `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Luottamusvälit

Lasketaan nyt luottamusvälit baseball-pelaajien painoille ja pituuksille. Käytämme koodia [tästä stackoverflow-keskustelusta](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypoteesin testaus

Tutkitaan eri rooleja baseball-pelaajien tietojoukossamme:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Testataan hypoteesia, että ykköspesänpelaajat ovat pidempiä kuin kakkospelin pelaajat. Yksinkertaisin tapa tehdä tämä on testata luottamusvälejä:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Voimme nähdä, että jaksot eivät mene päällekkäin.

Tilastollisesti oikein tapa todistaa hypoteesi on käyttää **Studentin t-testiä**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

`ttest_ind`-funktion palauttamat kaksi arvoa ovat:
* p-arvoa voidaan pitää todennäköisyytenä, että kahdella jakaumalla on sama keskiarvo. Tapauksessamme se on hyvin pieni, mikä tarkoittaa, että on vahvaa näyttöä siitä, että ykköspelaajat ovat pidempiä.
* t-arvo on normalisoidun keskiarvon erotuksen välivaihe, jota käytetään t-testissä, ja sitä verrataan tietylle luottamusarvolle asetettuun kynnysarvoon.


## Normaalijakauman simulointi keskeisen raja-arvolauseen avulla

Pythonin satunnaislukugeneraattori on suunniteltu antamaan meille tasajakauma. Jos haluamme luoda normaalijakaumalle generaattorin, voimme käyttää keskeistä raja-arvolausetta. Saadaksemme normaalisti jakautuneen arvon laskemme vain tasajakautuneesta otoksesta keskiarvon.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelaatio ja Evil Baseball Corp

Korrelaatio antaa meille mahdollisuuden löytää yhteyksiä datan sarjojen välillä. Leikkiesimerkissämme kuvitellaan, että on olemassa ilkeä baseball-yritys, joka maksaa pelaajilleen heidän pituutensa mukaan - mitä pidempi pelaaja on, sitä enemmän hän saa rahaa. Oletetaan, että peruspalkka on 1000 dollaria, ja lisäksi maksetaan bonus 0–100 dollaria pituuden perusteella. Otamme reaaliset MLB:n pelaajat ja laskemme heidän kuvitteelliset palkkansa:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Lasketaan nyt näiden sekvenssien kovarianssi ja korrelaatio. `np.cov` antaa meille niin sanotun **kovarianssimatriisin**, joka on laajennus kovarianssille useille muuttujille. Kovarianssimatriisin $M$ alkio $M_{ij}$ on korrelaatio syötemuuttujien $X_i$ ja $X_j$ välillä, ja diagonaaliarvot $M_{ii}$ ovat $X_{i}$ variansseja. Vastaavasti `np.corrcoef` antaa meille **korrelaatiomatriisin**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Korrelaatio, joka on yhtä suuri kuin 1, tarkoittaa, että kahden muuttujan välillä on vahva **lineaarinen yhteys**. Voimme havaita lineaarisen yhteyden visuaalisesti piirtämällä yhden arvon toista vastaan:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Katsotaanpa, mitä tapahtuu, jos suhde ei ole lineaarinen. Oletetaan, että yrityksemme päätti piilottaa ilmeisen lineaarisen riippuvuuden pituuden ja palkkojen välillä ja lisäsi kaavaan jonkin ei-lineaarisuuden, kuten `sin`:in:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Tässä tapauksessa korrelaatio on hieman pienempi, mutta se on silti melko korkea. Nyt, tehdäksemme suhteesta vielä vähemmän ilmeisen, saatamme haluta lisätä hieman ylimääräistä satunnaisuutta lisäämällä palkan arvoon satunnaisen muuttujan. Katsotaan mitä tapahtuu:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Arvaatko, miksi pisteet asettuvat pystysuoriksi viivoiksi näin?

Olemme havainneet korrelaation keinotekoisesti suunnitellun käsitteen, kuten palkan, ja havaintomuutujan *pituus* välillä. Tarkastellaan myös, korreloivatko kaksi havaittua muuttujaa, kuten pituus ja paino:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Valitettavasti emme saaneet mitään tuloksia – vain joitakin outoja `nan`-arvoja. Tämä johtuu siitä, että joitakin sarjan arvoja ei ole määritelty, ja ne esitetään arvolla `nan`, mikä aiheuttaa myös operaation tuloksen olevan määrittelemätön. Katsomalla matriisia voimme nähdä, että `Weight` on ongelmallinen sarake, koska korkeusarvojen itsekorrelaatio on laskettu.

> Tämä esimerkki osoittaa **aineiston valmistelun** ja **puhdistuksen** tärkeyden. Ilman asianmukaista aineistoa emme voi laskea mitään.

Käytetään `fillna`-metodia täyttämään puuttuvat arvot ja lasketaan korrelaatio:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Todellakin on olemassa korrelaatio, mutta ei niin vahva kuin meidän keinotekoisessa esimerkissä. Itse asiassa, jos katsomme hajontakaaviota, jossa toinen arvo asetetaan toisen vastaan, suhde olisi paljon vähemmän ilmeinen:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Yhteenveto

Tässä muistikirjassa olemme oppineet suorittamaan peruslaskutoimituksia datalla tilastollisten funktioiden laskemiseksi. Tiedämme nyt, kuinka käyttää vankkaa matemaattisten ja tilastollisten menetelmien kokonaisuutta hypoteesien todistamiseen sekä kuinka laskea luottamusvälit mielivaltaisille muuttujille annetun datan otoksen perusteella.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:
Tämä asiakirja on käännetty tekoälypohjaisella käännöspalvelulla [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta ole hyvä ja huomioi, että konekäännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäiskielellä tulee pitää virallisena lähteenä. Tärkeitä tietoja varten suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinymmärryksistä tai virhetulkinnoista.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
